In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ltim_c1_adb;
CREATE SCHEMA IF NOT EXISTS ltim_c1_adb.bronze;
CREATE SCHEMA IF NOT EXISTS ltim_c1_adb.silver;
CREATE SCHEMA IF NOT EXISTS ltim_c1_adb.gold;
CREATE SCHEMA IF NOT EXISTS ltim_c1_adb.control;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7411492579457338>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'CREATE CATALOG IF NOT EXISTS ltim_c1_adb;\nCREATE SCHEMA IF NOT EXISTS ltim_c1_adb.bronze;\nCREATE SCHEMA IF NOT EXISTS ltim_c1_adb.silver;\nCREATE SCHEMA IF NOT EXISTS ltim_c1_adb.gold;\nCREATE SCHEMA IF NOT EXISTS ltim_c1_adb.control;\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_S

In [0]:
%sql
DROP TABLE silver.shipments

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS ltim_c1_adb.bronze.raw;

In [0]:
%sql
DROP TABLE IF EXISTS control.processed_files;
CREATE TABLE IF NOT EXISTS control.processed_files (
    file_name STRING,
    processed_ts TIMESTAMP
)
USING delta;

In [0]:
from pyspark.sql import functions as F
bronze_path = "/Volumes/ltim_c1_adb/bronze/raw/"

all_files_df = (
    spark.createDataFrame(dbutils.fs.ls(bronze_path))
    .filter(F.col("name").endswith(".csv"))
    .select(F.col("name").alias("file_name"))
)
all_files_df.display()

file_name
shipment_capstone.csv
shipments_daily.csv


In [0]:
processed_df = spark.table("control.processed_files")

new_files_df = all_files_df.join(
    processed_df,
    on="file_name",
    how="left_anti"
)
new_files_df.display()

file_name
shipments_daily.csv


In [0]:

from pyspark.sql import functions as F
from pyspark.sql.window import Window

new_files = [row.file_name for row in new_files_df.collect()]

for file_name in new_files:

    file_path = f"{bronze_path}/{file_name}"

    # Read CSV
    df = (
        spark.read
            .format("csv")
            .option("header", "true")
            .load(file_path)
    )

    # Ingest metadata
    df = (
        df
        .withColumn("ingest_ts", F.current_timestamp())
        .withColumn("ingest_date", F.current_date())
        .withColumn("source_file", F.lit(file_name))
    )

    # Normalize columns
    for c in df.columns:
        df = df.withColumnRenamed(c, c.lower().replace(" ", "_"))

    # Trim + case
    for c in ["carrier_id", "delivery_status"]:
        df = df.withColumn(c, F.upper(F.trim(F.col(c))))

    # Cast types
    df = (
        df
        .withColumn("shipment_cost", F.col("shipment_cost").cast("double"))
        .withColumn("shipment_date", F.to_date("shipment_date"))
        .withColumn("delivery_date", F.to_date("delivery_date"))
    )

    # Empty → null
    df = df.select([
        F.when(F.trim(F.col(c)) == "", None).otherwise(F.col(c)).alias(c)
        for c in df.columns
    ])

    # Dedup within file
    w = Window.partitionBy("shipment_id").orderBy(F.col("ingest_ts").desc())
    df = df.withColumn("rn", F.row_number().over(w)).filter("rn=1").drop("rn")

    # UNKNOWN carrier
    df = df.withColumn("carrier_id", F.coalesce(F.col("carrier_id"), F.lit("UNKNOWN")))

    # Cost rounding
    df = df.withColumn("shipment_cost", F.round(F.col("shipment_cost"), 2))

    # Outlier detection
    median_cost = df.approxQuantile("shipment_cost", [0.5], 0.01)[0]
    df = df.withColumn("dq_cost_outlier", F.col("shipment_cost") > median_cost * 10)

    # DQ flags
    df = (
        df
        .withColumn("dq_cost_null", F.col("shipment_cost").isNull())
        .withColumn("dq_cost_negative", F.col("shipment_cost") < 0)
        .withColumn("dq_cost_zero", F.col("shipment_cost") == 0)
        .withColumn("dq_date_anomaly", F.col("shipment_date") > F.col("delivery_date"))
        .withColumn(
            "dq_invalid_status",
            ~F.col("delivery_status").isin("DELIVERED","IN_TRANSIT","CANCELLED")
        )
    )

    # Score + severity
    df = df.withColumn(
        "dq_score",
        F.expr("""
            int(dq_cost_null) +
            int(dq_cost_negative) +
            int(dq_cost_zero) +
            int(dq_cost_outlier) +
            int(dq_date_anomaly) +
            int(dq_invalid_status)
        """)
    )

    df = df.withColumn(
        "dq_severity",
        F.when(F.col("dq_score") >= 2, "CRITICAL")
             .when(F.col("dq_score") == 1, "MAJOR")
             .otherwise("MINOR")
    )

    df = df.withColumn("is_quarantine", F.col("dq_severity") == "CRITICAL")

    # Append to Silver
    df.write \
      .format("delta") \
      .mode("append") \
      .saveAsTable("silver.shipments")

    # Mark file as processed
    spark.createDataFrame(
        [(file_name,)],
        ["file_name"]
    ).withColumn(
        "processed_ts", F.current_timestamp()
    ).write.mode("append").saveAsTable("control.processed_files")


In [0]:
%sql
SELECT COUNT(*) AS total_records
FROM silver.shipments;

total_records
2506000


In [0]:
%sql
SELECT 
    COUNT(DISTINCT shipment_id) AS distinct_shipments
FROM silver.shipments;


distinct_shipments
2503000


In [0]:
%sql
select * FROM control.processed_files

file_name,processed_ts
shipment_capstone.csv,2026-02-09T05:17:20.157Z
shipments_daily.csv,2026-02-09T05:24:36.545Z


In [0]:
%sql
SELECT 
    source_file,
    COUNT(*) AS records_loaded
FROM silver.shipments
GROUP BY source_file
ORDER BY source_file;


source_file,records_loaded
shipment_capstone.csv,2503000
shipments_daily.csv,3000


In [0]:
%sql
SELECT COUNT(*) AS files_processed
FROM control.processed_files;


files_processed
2


In [0]:
%sql
SELECT COUNT(*) AS null_cost_records
FROM silver.shipments
WHERE dq_cost_null = true;


null_cost_records
100411


In [0]:
%sql
SELECT COUNT(*) AS date_anomaly_records
FROM silver.shipments
WHERE dq_date_anomaly = true;


date_anomaly_records
30


In [0]:
%sql
SELECT shipment_id, shipment_date, delivery_date
FROM silver.shipments
WHERE dq_date_anomaly = true
LIMIT 20;


shipment_id,shipment_date,delivery_date
L7337,2025-12-07,2025-12-06
L8389,2025-12-04,2025-12-01
L8615,2025-12-23,2025-12-22
L8688,2025-12-15,2025-12-13
L8985,2025-12-04,2025-12-01
L9689,2025-12-26,2025-12-23
L9718,2025-12-15,2025-12-13
L9830,2025-12-09,2025-12-06
L7373,2025-12-23,2025-12-21
L7773,2025-12-17,2025-12-15


In [0]:
%sql
SELECT 
    dq_severity,
    COUNT(*) AS record_count
FROM silver.shipments
GROUP BY dq_severity
ORDER BY dq_severity;


dq_severity,record_count
CRITICAL,14
MAJOR,35589
MINOR,2470397


In [0]:
%sql
SELECT
    COUNT(*) AS total_records,
    SUM(CASE WHEN dq_severity = 'CRITICAL' THEN 1 ELSE 0 END) AS critical_records,
    SUM(CASE WHEN dq_severity = 'MAJOR' THEN 1 ELSE 0 END) AS major_records,
    SUM(CASE WHEN dq_severity = 'MINOR' THEN 1 ELSE 0 END) AS minor_records
FROM silver.shipments;


total_records,critical_records,major_records,minor_records
2506000,14,35589,2470397
